In [0]:
%run ../utils/common

In [0]:

dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_sale_price_site 
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS
SELECT
  kappl AS app,
  kschl AS cond_type,
  vkorg AS sale_org,
  vtweg AS distribution_channel,
  werks AS store_id,
  matnr AS product_id,
  vrkme AS sale_unit,
  datbi AS valid_to,
  recordmode AS record_mode,
  datab AS valid_from,
  knumh AS cond_no
FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_vccmmrb27
WHERE proc_date = (SELECT MAX(proc_date) FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_vccmmrb27)
QUALIFY ROW_NUMBER() OVER (PARTITION BY kappl, kschl, vkorg, vtweg, werks, matnr, vrkme, datbi ORDER BY proc_date) = 1;
""")